In [8]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/MyDrive/work_space/Project/ABFR')

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from util.module import *
import torch.optim as optim

df = pd.read_csv('./Data/house-prices-advanced-regression-techniques/train.csv')
numeric_df = df.select_dtypes(include=['int', 'float'])

In [2]:
numeric_df = numeric_df.dropna(axis=0)
numeric_df.reset_index(inplace=True, drop=True)
test_df = numeric_df.iloc[:, 1:23]

array_df = np.array(test_df)
array_df = np.round(array_df, 3)

fa = Factor_attention(array_df)
fa.col_to_vec(threshold = 0.5)

In [5]:
model = Attention(n_factor= fa.n_factors, info_dim = fa.dim_info)
model(fa)

tensor([[ 6.0000e+01,  4.7895e+03,  5.0000e+00,  ..., -2.9092e+03,
          0.0000e+00,  1.0000e+00],
        [ 2.0000e+01,  5.0350e+03,  8.0000e+00,  ..., -9.6239e+02,
          1.0000e+00,  1.0000e+00],
        [ 6.0000e+01,  4.8939e+03,  5.0000e+00,  ..., -2.9907e+03,
          0.0000e+00,  1.0000e+00],
        ...,
        [ 7.0000e+01,  5.3590e+03,  9.0000e+00,  ..., -3.9552e+03,
          0.0000e+00,  1.0000e+00],
        [ 2.0000e+01,  4.6848e+03,  6.0000e+00,  ..., -8.2216e+02,
          0.0000e+00,  1.0000e+00],
        [ 2.0000e+01,  5.0077e+03,  6.0000e+00,  ..., -9.5928e+02,
          0.0000e+00,  1.0000e+00]], dtype=torch.float64,
       grad_fn=<CatBackward0>)

In [12]:
model = Attention(n_factor= fa.n_factors, info_dim = fa.dim_info)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 모델 학습
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(fa)
    loss = criterion(outputs, model(fa))
    loss.backward()
    optimizer.step()
    

In [15]:
model(fa).shape

torch.Size([1121, 12])

In [17]:
class FactorAnalysis(nn.Module):
    def __init__(self, num_vars, num_factors):
        super(FactorAnalysis, self).__init__()
        self.loadings = nn.Parameter(torch.randn(num_vars, num_factors))
        self.psi = nn.Parameter(torch.ones(num_vars))  # Uniquenesses, or specific variances
    
    def forward(self):
        # Reconstruct covariance matrix based on FA
        return torch.mm(self.loadings, self.loadings.t()) + torch.diag(self.psi)
    
def em_step(model, data):
    # Covariance matrix
    S = torch.mm(data.t(), data) / data.shape[0]
    
    # E-step
    L_psi_inv = torch.mm(model.loadings.t(), torch.inverse(torch.diag(model.psi)))
    M = torch.inverse(torch.eye(model.loadings.shape[1]) + torch.mm(L_psi_inv, model.loadings))
    E_h = torch.mm(torch.mm(M, model.loadings.t()), torch.inverse(torch.diag(model.psi)))
    E_hhT = M + torch.mm(E_h, torch.mm(data.t(), data))
    
    # M-step
    model.loadings.data = torch.mm(torch.mm(S, E_h.t()), torch.inverse(E_hhT))
    psi_diag = torch.diag(S - torch.mm(model.loadings, torch.mm(E_h, S)))
    model.psi.data = psi_diag / data.shape[0]

data = model(fa)

fa_model = FactorAnalysis(data.shape[-1], data.shape[-1])

num_epochs = 100
for epoch in range(num_epochs):
    em_step(fa_model, data)
    if (epoch + 1) % 10 == 0:
        loss = torch.norm(data - fa_model())
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


RuntimeError: inverse: LAPACK library not found in compilation

In [20]:
# Helper functions for EM-based FA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def e_step(X, loadings, unique_variances):
    """
    E-step of the EM algorithm for FA.
    """
    inv_diag_unique_variances = 1.0 / unique_variances
    inv_sigma = torch.diag(inv_diag_unique_variances) - torch.mm(
        torch.mm(loadings.t(), torch.diag(inv_diag_unique_variances)),
        torch.mm(loadings, torch.inverse(torch.eye(loadings.shape[1], device=device) + 
                                         torch.mm(loadings.t(), torch.mm(torch.diag(inv_diag_unique_variances), loadings))))
    )
    
    latent_mean = torch.mm(torch.mm(X, torch.diag(inv_diag_unique_variances)), torch.mm(loadings, inv_sigma))
    
    latent_covariance = torch.eye(loadings.shape[1], device=device) - torch.mm(
        torch.mm(loadings.t(), torch.diag(inv_diag_unique_variances)),
        torch.mm(loadings, inv_sigma)
    ) + torch.mm(inv_sigma, torch.mm(latent_mean.t(), latent_mean))
    
    return latent_mean, latent_covariance

def m_step(X, latent_mean, latent_covariance):
    """
    M-step of the EM algorithm for FA.
    """
    loadings = torch.mm(torch.mm(X.t(), latent_mean), torch.inverse(latent_covariance))
    
    diag_residual = torch.diag(torch.mm(X.t(), X)) - torch.diag(torch.mm(loadings, torch.mm(latent_mean.t(), X)))
    unique_variances = diag_residual / X.shape[0]
    
    return loadings, unique_variances

def em_factor_analysis(X, n_components, max_iter=100):
    """
    Factor Analysis using the EM algorithm.
    """
    n_features = X.shape[1]
    
    # Initialize parameters
    loadings = torch.randn(n_features, n_components, device=device)
    unique_variances = torch.ones(n_features, device=device)
    
    for i in range(max_iter):
        # E-step
        latent_mean, latent_covariance = e_step(X, loadings, unique_variances)
        
        # M-step
        loadings, unique_variances = m_step(X, latent_mean, latent_covariance)
    
    return loadings, unique_variances
def varimax_rotation(loadings, gamma=1.0, max_iter=100, tolerance=1e-6):
    """
    Apply Varimax rotation to the loadings matrix.
    """
    n, m = loadings.shape
    rotated_loadings = loadings.clone()
    rotation_matrix = torch.eye(m, device=device)
    
    for _ in range(max_iter):
        d = torch.mm(rotated_loadings, torch.mm(torch.diag(torch.sum(rotated_loadings ** 2, dim=0) ** (-gamma/2)), rotated_loadings.t()))
        u, s, v = torch.svd(torch.mm(d, rotated_loadings - (1.0 / n) * torch.mm(rotated_loadings, torch.diag(torch.sum(rotated_loadings, dim=0)))))
        rotation_matrix_new = torch.mm(u, v)
        rotated_loadings = torch.mm(loadings, rotation_matrix_new)
        
        if torch.norm(rotation_matrix_new - rotation_matrix) < tolerance:
            break
        
        rotation_matrix = rotation_matrix_new
    
    return rotated_loadings

# Apply EM-based FA and Varimax rotation
loadings, unique_variances = em_factor_analysis(model(fa), model(fa).shape[-1], max_iter=100)
rotated_loadings = varimax_rotation(loadings)

rotated_loadings


RuntimeError: inverse: LAPACK library not found in compilation

In [21]:
class TorchFactorAnalysis:
    def __init__(self):
        self.loadings = None
        self.rotation_matrix = None
        self.rotated_loadings = None
        self.factor_score = None
        self.factor_score_coefficent = None
        self.cov_mat = None
        self.explained_variance = None
        
    def fit(self, X, num_factor=2, rotation_type='varimax', k=4, max_iter=1000, eps=1e-05):
        assert rotation_type in ['varimax', 'promax']
        
        self.get_loadings(X, num_factor)
        L = self.loadings
        
        self.get_rotated_loadings(L, rotation_type=rotation_type, 
                                  max_iter=max_iter, eps=eps, k=k)
        
        self.get_factor_score(X)
        return self
    
    def get_rotated_loadings(self, L, rotation_type, k, max_iter, eps):
        if rotation_type == 'varimax':
            R, rotated_L = self._varimax(L, max_iter=max_iter, eps=eps)
            self.rotation_matrix = R
            self.rotated_loadings = rotated_L
        else:
            R, rotated_L = self._promax(L, k=k, max_iter=max_iter, eps=eps)
            self.rotation_matrix = R
            self.rotated_loadings = rotated_L
            
    def _varimax(self, L, max_iter=1000, eps=1e-05):
        R = torch.eye(L.shape[1], device=device)
        d_old = 0 
        p = L.shape[0]
        for i in range(max_iter):
            L_star = torch.mm(L, R)
            Q = torch.mm(L.T, L_star ** 3 - torch.mm(L_star, torch.diag(torch.diag(torch.mm(L_star.T, L_star)))/p))
            U, S, V = torch.svd(Q)
            R = torch.mm(U, V.T)
            d_new = torch.sum(S)
            if d_new < d_old * (1 + eps):
                break
            else:
                d_old = d_new
        return R, L_star        
        
    def _promax(self, L, k=4, max_iter=1000, eps=1e-5):
        _, L_star = self._varimax(L, max_iter=max_iter, eps=eps)
        P = L_star * (torch.abs(L_star) ** (k-1))
        L_inv = torch.inverse(torch.mm(L_star.T, L_star))
        R = torch.mm(torch.mm(L_inv, L_star.T), P)
        R = R/torch.sqrt(torch.sum(R ** 2, dim=0))
        L_final = torch.mm(L_star, R)
        return R, L_final
        
    def get_loadings(self, X, num_factor):
        cov_mat = torch.mm((X - torch.mean(X, dim=0)).T, (X - torch.mean(X, dim=0))) / (X.shape[0]-1)
        self.cov_mat = cov_mat
        eigen_values, eigen_vectors = torch.eig(cov_mat, eigenvectors=True)
        eigen_values = eigen_values[:num_factor, 0]
        eigen_vectors = eigen_vectors[:, :num_factor]
        loadings = torch.sqrt(eigen_values).unsqueeze(1) * eigen_vectors
        self.loadings = loadings
        self.explained_variance = torch.sum(eigen_values) / X.shape[1]
        
    def get_factor_score(self, X):
        L = self.rotated_loadings
        S_inv = torch.inverse(self.cov_mat)
        factor_score = torch.mm(torch.mm(L.T, S_inv), (X - torch.mean(X, dim=0)).T)
        self.factor_score_coefficent = (torch.mm(L.T, S_inv)).T
        self.factor_score = factor_score.T


In [23]:
temp_fa = TorchFactorAnalysis().fit(model(fa), num_factor=model(fa).shape[-1], rotation_type='promax')




C:\Users\Jang\AppData\Local\Temp\ipykernel_5480\3683534337.py:61: UserWarning: torch.eig is deprecated in favor of torch.linalg.eig and will be removed in a future PyTorch release.
torch.linalg.eig returns complex tensors of dtype cfloat or cdouble rather than real tensors mimicking complex tensors.
L, _ = torch.eig(A)
should be replaced with
L_complex = torch.linalg.eigvals(A)
and
L, V = torch.eig(A, eigenvectors=True)
should be replaced with
L_complex, V_complex = torch.linalg.eig(A) (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:3427.)
  eigen_values, eigen_vectors = torch.eig(cov_mat, eigenvectors=True)


RuntimeError: Calling torch.eig on a CPU tensor requires compiling PyTorch with LAPACK. Please use PyTorch built with LAPACK support.